### Creating your first graph and running it in a session

In [9]:
import tensorflow as tf

In [3]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [6]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [11]:
sess.close()

In [15]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval() # session is automaticaly closed

In [16]:
init = tf.global_variables_initializer() #prepare an init node

with tf.Session() as sess:
    init.run() # actually initialize all the variales
    restuly = f.eval()

In [17]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


In [18]:
sess.close()

### Managing Graphs

In [21]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [23]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [24]:
x2.graph is graph

True

In [25]:
x2.graph is tf.get_default_graph()

False

### Lifecycle of a node value

In [28]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


If we want to evaluate y and z efficiently without evaluating w and x twice as in the prvious code yo must ask TensorFlow to evaluate both y and z in just one graph run, as shown in the following code:

In [32]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


### Linear Regression with TensorFlow

In [33]:
import numpy as np

In [35]:
from sklearn.datasets import fetch_california_housing

In [48]:
housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [42]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32,name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT),y)

In [43]:
with tf.Session() as sess:
    theta_value = theta.eval()

### Using Batch Gradient Descent

Gradient Descent requires scaling the feature vectors first. We could do this using TF, but let's just use Scikit-Learn for now.



In [56]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [57]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


### Manually Computing the Gradients

In [61]:
n_epochs = 1000
learning_rate = 0.01

In [63]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32,name="y")

theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred -y
mse =tf.reduce_mean(tf.square(error),name ="mse")
gradients = 2/m *tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate *gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch: ", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch:  0 MSE = 11.627701
Epoch:  100 MSE = 0.87371874
Epoch:  200 MSE = 0.6434772
Epoch:  300 MSE = 0.6077646
Epoch:  400 MSE = 0.58466274
Epoch:  500 MSE = 0.5680381
Epoch:  600 MSE = 0.55601776
Epoch:  700 MSE = 0.5473198
Epoch:  800 MSE = 0.54102385
Epoch:  900 MSE = 0.5364632


### AutoDiff

In [67]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [68]:
gradients = tf.gradients(mse, [theta])[0]

In [69]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.7544272
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727796
Epoch 300 MSE = 0.5585005
Epoch 400 MSE = 0.54906934
Epoch 500 MSE = 0.5422877
Epoch 600 MSE = 0.5373788
Epoch 700 MSE = 0.5338219
Epoch 800 MSE = 0.5312427
Epoch 900 MSE = 0.52937055
Best theta:
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


In [70]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [71]:
my_func(0.2, 0.3)

-0.21253923284754914

In [72]:
a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")
for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)

grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

In [73]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.21253741
[-1.1388494, 0.19671395]
